## Búsqueda binaria con índice invertido (BSII)

Importamos librerías de utilidad. Entre ellas destaca `nltk`, que será la principal encargada de la limpieza y el preprocesamiento del texto.

In [1]:
import os
from pathlib import Path
from typing import List
from collections import Counter
import xml.etree.ElementTree as ET

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd

# Descargar recursos necesarios de NLTK
nltk.download('punkt')  # Tokenizador
nltk.download('stopwords')  # Stopwords

# Configuración de NLTK
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
def token_preprocessing(tokens: List[str]) -> List[str]:
    """
    Realiza el preprocesamiento de texto, aplicando las siguientes transformaciones:
    1. Convierte todos los tokens a minúsculas.
    2. Elimina stopwords.
    3. Elimina tokens que no son palabras (puntuación, números, etc.).
    4. Aplica stemming con PorterStemmer.

    Args:
        tokens (List[str]): Lista de tokens a preprocesar.

    Returns:
        List[str]: Lista de tokens preprocesados.
    """
    # Pasa los tokens a minúsculas
    tokens = [word.lower() for word in tokens]

    # Elimina stopwords (asume texto en inglés)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Filtrar tokens no alfabéticos (palabras que no empiezan con una letra)
    tokens = [word for word in tokens if 'a' <= word[0] <= 'z']

    # Aplica stemming
    ps = PorterStemmer()
    return [ps.stem(word) for word in tokens]

def text_preprocessing(text: str) -> List[str]:
    """
    Preprocesa un texto mediante la tokenización y el preprocesamiento de tokens.

    Args:
        text (str): Texto a preprocesar.

    Returns:
        List[str]: Lista de tokens preprocesados.
    """
    # Tokenizar el texto usando el tokenizador de nltk
    tokens = word_tokenize(text)
    
    # Preprocesar los tokens
    return token_preprocessing(tokens)

In [3]:
text_preprocessing(
    "William Beaumont: Physiology of digestion Image Source.  On November 21, 1785, US-American surgeon William Beaumont was born.")

['william',
 'beaumont',
 'physiolog',
 'digest',
 'imag',
 'sourc',
 'novemb',
 'us-american',
 'surgeon',
 'william',
 'beaumont',
 'born']

In [4]:
class Document:
    def __init__(self, text: str, name: str = 'nameless'):
        """
        Crea un objeto del tipo Document.

        Args:
            text (str): El contenido del documento.
            name (str): El nombre del documento. Por defecto es 'nameless'.
        """
        self.text = text
        self.name = name

        # Preprocesar el texto y contar la frecuencia de cada token
        counter = Counter(text_preprocessing(text))
        
        # Almacenar los conteos de términos como una Serie de pandas
        self.term_counts = pd.Series(counter.values(), index=counter.keys())

    def __repr__(self):
        """
        Representación formal del objeto Document.
        """
        return str(self)

    def __str__(self):
        """
        Representación informal del objeto Document, devuelve el nombre del documento.
        """
        return self.name

In [5]:
def load_docs(docs_folder_path: Path) -> List[Document]:
    """
    Carga los documentos (con extensión .naf) en una carpeta especificada y crea objetos Document para cada uno.

    Args:
        docs_folder_path (Path): Ruta a la carpeta que contiene los archivos .naf.

    Returns:
        List[Document]: Una lista de objetos Document.
    """
    # Encuentra todos los archivos con extensión .naf en la carpeta especificada
    files = [f for f in os.listdir(docs_folder_path) if f.endswith('.naf')]

    # Lista para almacenar los documentos cargados
    docs = []

    # Procesar cada archivo .naf
    for file in files:
        # Parsea el archivo XML
        tree = ET.parse(os.path.join(docs_folder_path, file))
        root = tree.getroot()

        # Extrae el texto crudo del documento
        raw_text = root.find('.//raw').text

        # Extrae el nombre del archivo, omitiendo la extensión
        name = file.split('.')[1]

        # Crea un objeto Document y lo añade a la lista
        docs.append(Document(raw_text, name))

    return docs

In [6]:
all_docs = load_docs(Path("./data/docs-raw-texts"))

In [10]:
class BSII:
    def __init__(self, docs: List[Document]):
        """
        Inicializa el índice invertido utilizando una lista de documentos.

        Args:
            docs (List[Document]): Lista de objetos Document a indexar.
        """
        self.docs = docs
        self.inverse_index = {}

        # Construir el índice invertido
        for doc in self.docs:

            # Procesa cada término del documento
            for term in doc.term_counts.index:

                # Si el término no está en el índice invertido, lo agrega
                if term not in self.inverse_index:
                    self.inverse_index[term] = set()

                # Añade el documento al listing del término
                self.inverse_index[term].add(doc)

    def search(self, query_document: Document = None, excluded_query_document: Document = None) -> set:
        """
        Realiza una búsqueda en el índice invertido, incluyendo o excluyendo documentos según los términos. Note que procesamos los queries como objetos de tipo Documento.

        Args:
            query_document (Document, opcional): Documento cuyas palabras clave se utilizarán para buscar documentos relevantes. Esto implementa el AND.
            excluded_query_document (Document, opcional): Documento cuyas palabras clave se utilizarán para excluir documentos de los resultados. Esto es equivalente a un NOT.

        Returns:
            set: Un conjunto de documentos que cumplen con los criterios de la búsqueda.
        """
        relevant_docs = set(self.docs)

        # Incluir documentos relevantes según el query_document
        if query_document is not None:
            for term in query_document.term_counts.index:
                if term in self.inverse_index:
                    relevant_docs.intersection_update(self.inverse_index[term])

        # Excluir documentos según el excluded_query_document
        if excluded_query_document is not None:
            for term in excluded_query_document.term_counts.index:
                if term in self.inverse_index:
                    relevant_docs.difference_update(self.inverse_index[term])

        return relevant_docs

    def evaluate_search(self, queries: List[Document], output_path: Path):
        """
        Evalúa las consultas de búsqueda y escribe los resultados en un archivo de salida.

        Args:
            queries (List[Document]): Lista de documentos a utilizar como consultas.
            output_path (Path): Ruta del archivo donde se guardarán los resultados.
        """
        with open(output_path, 'w') as output_file:
            for query in queries:
                relevant_docs = self.search(query_document=query)
                # Escribir el nombre de la consulta seguido de los nombres de los documentos relevantes
                output_file.write(f"{query.name}\t{','.join(doc.name for doc in relevant_docs)}\n")

In [12]:
bsii = BSII(all_docs)
bsii.search(query_document=Document('Physiology'), excluded_query_document=Document('Swiss'))

{d001, d046, d062, d120, d133, d191, d261, d286, d294, d314}

Qué hacemos con términos que no están en el índice invertido para las búsquedas? Por defecto ahora los está ignorando simplemente:

In [14]:
bsii.search(query_document=Document('Physiology Esternocleidomastoideo'), excluded_query_document=Document('Swiss'))

{d001, d046, d062, d120, d133, d191, d261, d286, d294, d314}

In [13]:
all_queries = load_docs(Path("./data/queries-raw-texts"))
bsii.evaluate_search(all_queries, 'data/BSII-AND-queries_result')